In [1]:
from module_imports import *
from download_data import *
from import_data import *
from larochelle_nn import *

%matplotlib inline

download_quandl()

stock_df, prediction_df = pd.DataFrame(), pd.DataFrame()
pred_tickers = []
source = "Q"
binarize = True
gt = 0
lt = 10.0
vol = 0
if source == "Q":
    stock_df, prediction_df, pred_tickers = get_quandl_data(binarize=True, gt=gt, lt=lt, vol=vol)
elif source == "G":
    stock_df, prediction_df, pred_tickers = get_goog_data(binarize=True, gt=gt, lt=lt, vol=vol)
    
Y = stock_df['label'].values
Y = Y.reshape(Y.shape[0], 1)

X_df = stock_df.drop('label', axis=1)
X = X_df.values

print X.shape, Y.shape
X_df.tail()


96 total tickers to start

removed: AAVL
removed: APTO
removed: ATRA
removed: BLFS
removed: CAPN
removed: CASI
removed: CDTX
removed: CGNT
removed: CHRS
removed: CLLS
removed: DBVT
removed: EDAP
removed: GENE
removed: INCR
removed: JUNO
removed: KITE
removed: LOXO
removed: TENX

78 available tickers:
['ABIO', 'ACOR', 'ADMA', 'AERI', 'AFFX', 'AGEN', 'APPY', 'ARDM', 'ARIA', 'ARNA', 'ARWR', 'AXDX', 'AXGN', 'BABY', 'BASI', 'BCLI', 'BCRX', 'BGMD', 'BIIB', 'BLUE', 'BOTA', 'BRKR', 'CBLI', 'CBMG', 'CBMX', 'CBPO', 'CGEN', 'CLDN', 'CLDX', 'CNMD', 'COHR', 'CPHD', 'CPRX', 'CRIS', 'CUTR', 'CYBX', 'CYNO', 'CYTR', 'DARA', 'DRAD', 'DSCO', 'DYAX', 'ECTE', 'ECYT', 'ELOS', 'ENZN', 'ESMC', 'ETRM', 'EXAS', 'EXEL', 'FATE', 'FEIC', 'FLDM', 'FONR', 'GILD', 'GNCA', 'HALO', 'HSKA', 'IART', 'ICCC', 'IDRA', 'IDXX', 'ILMN', 'IMMU', 'IMRS', 'INCY', 'INO', 'IRIX', 'LPCN', 'MEIP', 'MNKD', 'OREX', 'PGNX', 'QLTI', 'RMTI', 'SGYP', 'THLD', 'TNXP']
(170974, 11) (170974, 1)


,200dravg,50dravg,Close,HL%,High,Low,OC%,Open,Volume,date,ticker
823,7.04130,8.5224,6.91,0.106259,7.60,6.87,-0.052126,7.29,138100,2015-08-19,78
824,7.03935,8.4868,6.66,0.060976,6.96,6.56,-0.029155,6.86,121400,2015-08-20,78
825,7.03830,8.4550,6.56,0.074303,6.94,6.46,-0.017964,6.68,166700,2015-08-21,78
826,7.03650,8.4026,6.20,0.120782,6.31,5.63,-0.009585,6.26,229200,2015-08-24,78
827,7.03310,8.3398,6.12,0.084437,6.55,6.04,-0.045242,6.41,150200,2015-08-25,78


In [ ]:
'''
# randomly balance labeled data
indices_Y_is_0 = np.where(Y == 0)[0]
print indices_Y_is_0.shape[0]
indices_Y_is_1 = np.where(Y == 1)[0]
print indices_Y_is_1.shape[0]

subset_indices_Y_is_0 = np.random.choice(indices_Y_is_0, indices_Y_is_1.shape[0])
X_is_0 = X[subset_indices_Y_is_0]
Y_is_0 = Y[subset_indices_Y_is_0]
X_is_1 = X[indices_Y_is_1]
Y_is_1 = Y[indices_Y_is_1]

X = np.concatenate((X_is_0,X_is_1))
Y = np.concatenate((Y_is_0,Y_is_1))
'''

In [ ]:
'''
# double positive examples
indices_Y_is_1 = np.where(Y == 1)[0]
print len(indices_Y_is_1)

X_is_1 = X[indices_Y_is_1]
Y_is_1 = Y[indices_Y_is_1]

print X_is_1.shape, Y_is_1.shape

iterations = (Y.shape[0] - Y_is_1.shape[0])/Y_is_1.shape[0]

for i in xrange(iterations):
    X = np.concatenate((X, X_is_1))
    Y = np.concatenate((Y, Y_is_1))
    
print X.shape, Y.shape
'''

In [ ]:
plt.hist(Y)
plt.show()

In [ ]:
vectorize_label = True
if vectorize_label == True:
    new_y = []
    positives = []
    for i in xrange(Y.shape[0]):
        if Y[i] == 0:
            new_y.append(np.array([[1],[0]]))
        elif Y[i] == 1:
            new_y.append(np.array([[0],[1]]))
    Y = np.array(new_y)

In [ ]:
X_train, X_vt, y_train, y_vt = train_test_split(X, Y, test_size=0.30, random_state=42)
X_validation, X_test, y_validation, y_test = train_test_split(X_vt, y_vt, test_size=0.50, random_state=42)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

In [ ]:
#for x,y in zip(X_train, y_train):
#    print x.reshape(x.shape[0],1)
#    print y.reshape(y.shape[0],1)
#    break

---

In [ ]:
h = 10

epochs = 800
Lambda = 0.01
Reg = 'L2'
alpha = 0.0001

W = []
b = []

W,b = NN_SGD(source, binarize, gt, lt, vol, balance_labeled_data, scaler, X_train, y_train, X_validation, y_validation, h, h, epochs, Lambda, Reg, alpha, plot=True)

---

In [ ]:
def loss(f_x,y):
    i = np.where(y == 1)[0][0]
    return -np.log(f_x[i]) # negative log-likelihood

def forward_prop(x, W1, b1, W2, b2, W3, b3):
    def sigm(z):
        return 1/(1+np.exp(-z))

    def softmax(z):
        return np.exp(z)/np.sum(np.exp(z))

    z1 = b1 + np.dot(W1,x)
    a1 = sigm(z1)

    z2 = b2 + np.dot(W2,a1)
    a2 = sigm(z2)

    z3 = b3 + np.dot(W3,a2)
    a3 = softmax(z3)

    f_x = a3
    return z1, a1, z2, a2, z3, a3, f_x

testing_losses = []
# testing
for x,y in zip(X_test, y_test):

    x = scaler.transform(x)
    x = x.reshape(x.shape[0],1)
    y = y.reshape(y.shape[0],1)

    z1, a1, z2, a2, z3, a3, f_x = forward_prop(x, W[0], b[0], W[1], b[1], W[2], b[2])

    testing_loss = np.round(loss(f_x,y),2)
    testing_losses.append(testing_loss)
    
    
print "Testing Loss:", np.mean(testing_losses)

In [ ]:
pred_df = prediction_df[prediction_df['label'].apply(np.isnan) == True]
pred_X = pred_df.drop(['label'], axis=1).values

In [ ]:
for i in xrange(pred_X.shape[0]):
    x = scaler.transform(pred_X[i])
    x = x.reshape(x.shape[0],1)
    
    z1, a1, z2, a2, z3, a3, f_x = forward_prop(x, W[0], b[0], W[1], b[1], W[2], b[2])
    
    print str(i).rjust(2), str(pred_tickers[i]).rjust(4), f_x[0], f_x[1]
    